In [49]:
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sqlite3 as sq
import sys
sys.path.append("../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
import requests
import io

Hey Joanna,   

I met with Chris Taylor, the Director of Forward Sumner yesterday. He asked if we had data for the following for Sumner Co.:  
1.	Building permits  
2.	Business closures  
Thank you!


In [50]:
conn = sq.connect('../../Pipeline-Housing-and-Urban-Development/Outputs/HUD.db')
sql_query = pd.read_sql('SELECT * FROM [SingleMulti_Historic]', conn)
data = pd.DataFrame(sql_query)
data.drop(columns = 'Source', inplace = True)
data = data.set_index('NAME').transpose()
data = data.rename(columns = namestocommon)
geos = ['Sumner County', 'GNRC', 'Portland', 'Gallatin', 'Hendersonville']
data = data[geos]
data = data.transpose().reset_index(drop = False)
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Time Frame')
cols.remove('GEO_ID')
data['Year'] = data['Year'].replace({'None': 0})
data[cols] = data[cols].astype(float)
hudann = data

In [51]:
hudann.head()

,NAME,Year,Time Frame,Units in All Multi-Family Structures,Units in All Multi-Family Structures % Change,Units in All Multi-Family Structures Change,Units in All Structures,Units in All Structures % Change,Units in All Structures Change,Units in Single-Family Structures,Units in Single-Family Structures % Change,Units in Single-Family Structures Change,GEO_ID
0,Sumner County,1980.0,None,105.0,NaN,NaN,712.0,NaN,NaN,607.0,NaN,NaN,0500000US47165
1,Sumner County,1981.0,None,32.0,NaN,NaN,455.0,NaN,NaN,423.0,NaN,NaN,0500000US47165
2,Sumner County,1982.0,None,49.0,NaN,NaN,460.0,NaN,NaN,411.0,NaN,NaN,0500000US47165
3,Sumner County,1983.0,None,723.0,NaN,NaN,1438.0,NaN,NaN,715.0,NaN,NaN,0500000US47165
4,Sumner County,1984.0,None,1093.0,NaN,NaN,1841.0,NaN,NaN,748.0,NaN,NaN,0500000US47165


In [52]:
hudann = hudann.loc[hudann['Time Frame'] == 'None']
hudann = hudann[['NAME', 'GEO_ID', 'Year', 'Units in All Multi-Family Structures', 'Units in Single-Family Structures', 'Units in All Structures']]

In [53]:
hudann.head()

,NAME,GEO_ID,Year,Units in All Multi-Family Structures,Units in Single-Family Structures,Units in All Structures
0,Sumner County,0500000US47165,1980.0,105.0,607.0,712.0
1,Sumner County,0500000US47165,1981.0,32.0,423.0,455.0
2,Sumner County,0500000US47165,1982.0,49.0,411.0,460.0
3,Sumner County,0500000US47165,1983.0,723.0,715.0,1438.0
4,Sumner County,0500000US47165,1984.0,1093.0,748.0,1841.0


In [54]:
# '1980','1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', 
#  '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', 
#  '2012', '2013', '2014', '2015', '2016', '2017', 

years = ['2018', '2019', '2020', '2021', '2022',  '2023']

In [60]:
#South Loop
data_appended = []
for i in years:
    url = 'https://www2.census.gov/econ/bps/Place/South%20Region/so{}a.txt'.format(i)
    t = requests.get(url).content
    col_names = ('Year','StateCode','6DigitID','CountyCode','CensusPlaceCode','PlaceFIPS','FIPSMCD','Pop','CSA','CBSA','Footnote','CentralCity',
             'ZipCode','Region','Division','NumberofMonthsRepresented','NAME','1UBuildings','1UUnits','1UValuation','2UBuildings','2UUnits','2UValuation',
             '3to4UBuildings','3to4UUnits','3to4UValuation','5+UBuildings','5+UUnits','5+UValuation') #set column names
    header = 2 #set to skip the header.. their column names sux
    #At this point decoded is not a file, it is a character string with the content from the web page. The next lines make it into a file.
    data = pd.read_csv(io.StringIO(t.decode('utf-8')), skiprows = header, 
                      header = None, names = col_names,
                       skipinitialspace = True, 
                      index_col = False,dtype = str)
    data_appended.append(data)
    
data_appended = pd.concat(data_appended)
south = data_appended
south = south.loc[south['StateCode'] == '47']
print('Go ahead')
#Note that the error is because I am not parsing the last few columns because they are duplicates of the first set of permitting numbers.

C:\Users\jmccall\AppData\Local\Temp\ipykernel_23572\3207582709.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(io.StringIO(t.decode('utf-8')), skiprows = header,
C:\Users\jmccall\AppData\Local\Temp\ipykernel_23572\3207582709.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(io.StringIO(t.decode('utf-8')), skiprows = header,
C:\Users\jmccall\AppData\Local\Temp\ipykernel_23572\3207582709.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(io.StringIO(t.decode('utf-8')), skiprows = header,
C:\Users\jmccall\AppData\Local\Temp\ipykernel_23572\3207582709.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.

Go ahead


C:\Users\jmccall\AppData\Local\Temp\ipykernel_23572\3207582709.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(io.StringIO(t.decode('utf-8')), skiprows = header,


In [61]:
south = south[['NAME', 'Year', '1UBuildings','1UUnits','2UBuildings','2UUnits',
             '3to4UBuildings','3to4UUnits','5+UBuildings','5+UUnits']]
cols = ['1UBuildings','1UUnits','2UBuildings','2UUnits',
             '3to4UBuildings','3to4UUnits','5+UBuildings','5+UUnits']
south[cols] = south[cols].astype(float)

In [62]:
thelist = [south['1UBuildings'], south['2UBuildings'], south['3to4UBuildings'], south['5+UBuildings']]
south['Total Buildings'] = sum(thelist)
south['Total Single Family Buildings'] = south['1UBuildings']
thelist = [south['2UBuildings'], south['3to4UBuildings'], south['5+UBuildings']]
south['Total Multifamily Buildings'] = sum(thelist)

thelist = [south['1UUnits'], south['2UUnits'], south['3to4UUnits'], south['5+UUnits']]
south['Total Units'] = sum(thelist)
south['Total Single Family Units'] = south['1UUnits']
thelist = [south['2UUnits'], south['3to4UUnits'], south['5+UUnits']]
south['Total Multifamily Units'] = sum(thelist)

south = south.drop(columns = cols)

In [64]:
#check out Coopertown town
CT = south.loc[(south['NAME'].str.contains('Portland'))]
CT

,NAME,Year,Total Buildings,Total Single Family Buildings,Total Multifamily Buildings,Total Units,Total Single Family Units,Total Multifamily Units
3054,Portland,2018,95.0,95.0,0.0,95.0,95.0,0.0
3043,Portland,2019,91.0,91.0,0.0,91.0,91.0,0.0
3044,Portland,2020,46.0,40.0,6.0,94.0,40.0,54.0
3042,Portland,2021,59.0,53.0,6.0,105.0,53.0,52.0
3037,Portland,2022,49.0,45.0,4.0,53.0,45.0,8.0
2992,Portland,2023,28.0,19.0,9.0,37.0,19.0,18.0


In [ ]:
CT = CT[['SurveyData', '1UBuildings', '1UUnits', '2UBuildings', '2UUnits', 3to4UBuildings, 3to4UUnits, 5UBuildings	5UUnits	5UValuation]]

In [65]:
port = hudann.loc[(hudann['NAME'] == 'Portland') & (hudann['Year'] > 2017)]

In [66]:
port

,NAME,GEO_ID,Year,Units in All Multi-Family Structures,Units in Single-Family Structures,Units in All Structures
2018,Portland,1600000US4760280,2018.0,0.0,95.0,95.0
2019,Portland,1600000US4760280,2019.0,0.0,91.0,91.0
2020,Portland,1600000US4760280,2020.0,54.0,40.0,94.0
2021,Portland,1600000US4760280,2021.0,52.0,53.0,105.0
2022,Portland,1600000US4760280,2022.0,8.0,45.0,53.0
2023,Portland,1600000US4760280,2023.0,18.0,19.0,37.0


CONCLUSION - THEY ARE THE SAME DATA!